# DataLab User Role Management

This notebook demonstrates how to set admin and user roles in DataLab using the administrative tasks from `tasks.py`. 

## Overview

DataLab has three user roles:
- **USER**: Default role with basic access
- **ADMIN**: Full administrative privileges 
- **MANAGER**: Intermediate role with some administrative capabilities

The role management is handled through:
1. Command-line invoke tasks (for server administrators)
2. Direct MongoDB operations (programmatic access)
3. Web interface (for admins through the UI)

## Key Functions from tasks.py

- `change_user_role()`: Changes a user's role by display name
- `manually_register_user()`: Creates a new user account with specified details

In [1]:
# Import required modules
import sys
import os
from bson import ObjectId

# Add the pydatalab path so we can import the modules
sys.path.append('/home/ian/datalab-deployment/datalab/pydatalab/src')

# Import DataLab modules
from pydatalab.models.utils import UserRole
from pydatalab.mongo import _get_active_mongo_client
from pydatalab.models.people import Identity, Person

print("Available User Roles:")
for role in UserRole:
    print(f"  - {role.name}: {role.value}")
    
print(f"\nUserRole enum members: {list(UserRole.__members__.keys())}")

Available User Roles:
  - USER: user
  - ADMIN: admin
  - MANAGER: manager

UserRole enum members: ['USER', 'ADMIN', 'MANAGER']


## Set User Admin - Using Invoke Tasks (Recommended for Server Administrators)

The `tasks.py` file provides invoke tasks that can be run from the command line. These are the safest and most straightforward methods for role management.

!! To execute the below ``invoke`` operations within the docker, you must use these flags before the commands: ``docker compose exec api uv run``!! 

e.g., 

```bash
# Change a user's role to admin
docker compose exec api uv run invoke admin.change-user-role --display-name="John Doe" --role=admin
```

### Command Line Usage Examples:

```bash
# Change a user's role to admin
invoke admin.change-user-role --display-name="John Doe" --role=admin

# Change a user's role to manager  
invoke admin.change-user-role --display-name="Jane Smith" --role=manager

# Change a user's role back to regular user
invoke admin.change-user-role --display-name="Bob Johnson" --role=user

# Register a new user manually
invoke admin.manually-register-user --display-name="New User" --contact-email="user@example.com"

# Register a new user with additional identities
invoke admin.manually-register-user --display-name="GitHub User" --contact-email="dev@example.com" --github-user-id=12345 --orcid="0000-0000-0000-0000"
```

### <span style="color:red">Note:</span>

If there are users of the same first or last names, you may encounter issues with ` Too many matches for display name `. To resolve this, simply search for the element of the name that do not overlap with anyone else:

```bash
$ docker compose exec api uv run invoke admin.change-user-role --display-name='Ian Yang' --role=admin
Bytecode compiled 11718 files in 296ms
Too many matches for display name 'Ian Yang': ['Ian Yang', 'Xi Yang']
exit status 1

$ docker compose exec api uv run invoke admin.change-user-role --display-name='Ian' --role=admin
Bytecode compiled 11718 files in 296ms
Found user: Ian Yang (6863896cce73d29aa19ee930)
Updated Ian to admin.
```

## Important Notes and Best Practices

### ⚠️ Security Warnings

1. **Admin Role Assignment**: Be very careful when assigning admin roles. Admins have full access to the system.
2. **Database Backups**: Always backup your MongoDB database before making bulk role changes.
3. **Production Safety**: Test role changes in a development environment first.

### 🔧 Database Storage Details

- **Users Collection**: `datalabvue.users` - stores user profile information
- **Roles Collection**: `datalabvue.roles` - stores role assignments (separate from users)
- **Default Role**: Users without entries in the roles collection default to "user" role

### 📋 Role Permissions

- **USER**: Basic access to create and manage their own items
- **MANAGER**: Can manage other users and has elevated permissions  
- **ADMIN**: Full system access, can manage all users and system settings

### 🛠️ Troubleshooting

1. **MongoDB Connection Issues**: Ensure MongoDB is running and the connection string is correct
2. **User Not Found**: Check that the display name matches exactly (case-sensitive)
3. **Permission Denied**: Only admins can change other users' roles
4. **Text Search Errors**: If text search fails, the system falls back to regex matching

### 📚 Command Line Alternatives

For production systems, prefer using the invoke tasks:

```bash
# List all available admin tasks
invoke --list admin

# Get help for a specific task
invoke admin.change-user-role --help

# Examples
invoke admin.change-user-role --display-name="John Doe" --role=admin
invoke admin.manually-register-user --display-name="New Admin" --contact-email="admin@example.com"
```

### 🔗 Related Files

- `/pydatalab/tasks.py` - Contains the invoke task definitions
- `/pydatalab/src/pydatalab/models/utils.py` - UserRole enum definition
- `/pydatalab/src/pydatalab/routes/v0_1/admin.py` - Web API endpoints for role management
- `/webapp/src/components/UserTable.vue` - Frontend user management interface

In [3]:
# TODO - Fix this: Utility Functions for User Management

def list_all_users_with_roles():
    """
    List all users and their current roles.
    Similar to what the admin interface shows.
    """
    client = _get_active_mongo_client()
    
    # Use aggregation pipeline to join users with their roles
    pipeline = [
        {
            "$lookup": {
                "from": "roles",
                "localField": "_id",
                "foreignField": "_id",
                "as": "role"
            }
        },
        {
            "$addFields": {
                "role": {
                    "$cond": {
                        "if": {"$eq": [{"$size": "$role"}, 0]},
                        "then": "user",  # Default role
                        "else": {"$arrayElemAt": ["$role.role", 0]},
                    }
                }
            }
        },
        {
            "$project": {
                "display_name": 1,
                "contact_email": 1, 
                "account_status": 1,
                "role": 1,
                "_id": 1
            }
        }
    ]
    
    users = list(client.datalabvue.users.aggregate(pipeline))
    
    if not users:
        print("No users found in the database.")
        return []
    
    print(f"Found {len(users)} users:")
    print("-" * 80)
    print(f"{'Display Name':<25} {'Email':<30} {'Role':<10} {'Status':<12}")
    print("-" * 80)
    
    for user in users:
        display_name = user.get('display_name', 'N/A')[:24]
        email = user.get('contact_email', 'N/A')[:29]
        role = user.get('role', 'user')
        status = user.get('account_status', 'unknown')[:11]
        print(f"{display_name:<25} {email:<30} {role:<10} {status:<12}")
    
    return users

def find_user_by_name(search_name: str):
    """
    Find users by display name (supports partial matching).
    """
    client = _get_active_mongo_client()
    
    # Search using text search if available, otherwise use regex
    try:
        matches = list(
            client.datalabvue.users.find(
                {"$text": {"$search": search_name}}, 
                projection={"_id": 1, "display_name": 1, "contact_email": 1}
            )
        )
    except:
        # Fallback to regex search if text search fails
        matches = list(
            client.datalabvue.users.find(
                {"display_name": {"$regex": search_name, "$options": "i"}}, 
                projection={"_id": 1, "display_name": 1, "contact_email": 1}
            )
        )
    
    if not matches:
        print(f"No users found matching: '{search_name}'")
        return []
    
    print(f"Found {len(matches)} user(s) matching '{search_name}':")
    for match in matches:
        print(f"  - {match['display_name']} ({match.get('contact_email', 'no email')}) [ID: {match['_id']}]")
    
    return matches

def get_user_role(user_id_or_name):
    """
    Get the current role of a user by ID or name.
    """
    client = _get_active_mongo_client()
    
    if isinstance(user_id_or_name, str) and len(user_id_or_name) == 24:
        # Assume it's an ObjectId string
        try:
            user_id = ObjectId(user_id_or_name)
        except:
            # If conversion fails, treat as name
            matches = find_user_by_name(user_id_or_name)
            if not matches:
                return None
            user_id = matches[0]['_id']
    else:
        # Treat as display name
        matches = find_user_by_name(user_id_or_name)
        if not matches:
            return None
        user_id = matches[0]['_id']
    
    # Get role
    role_doc = client.datalabvue.roles.find_one({"_id": user_id})
    if role_doc:
        return role_doc['role']
    else:
        return "user"  # Default role

# Example usage (commented out to avoid accidental execution)
list_all_users_with_roles()
find_user_by_name("Ian Yang")
get_user_role("Ian Yang")

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 1000.0ms, connectTimeoutMS: 1000.0ms), Timeout: 1.0s, Topology Description: <TopologyDescription id: 68889282b5a686d5fe980dcb, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 1000.0ms, connectTimeoutMS: 1000.0ms)')>]>

## Deactivate a User

The easiest way to deactivate a user is via the web interface when logged in as an admin. Programmatic access is currently not working...

## User Deletion (functionality currently not available) 
Notice that in the datalab schema, users are typically identified by `display_name` rather than username. Looking at the test fixtures in `conftest.py`, the user documents have fields like:

- `display_name`
- `contact_email`
- `account_status`

So your query should use `display_name` instead of username.

In [4]:
import sys
sys.path.append('/home/ian/datalab-deployment/datalab/pydatalab/src')

# Use datalab's mongo utilities that handle configuration automatically
from pydatalab.mongo import _get_active_mongo_client

try:
    # This will use the configured MONGO_URI from datalab's config
    client = _get_active_mongo_client()
    db = client.get_database()
    
    # Test the connection
    print("=== Connection successful ===")
    print(f"Database name: {db.name}")
    
    # List users
    print("\n=== All users in database ===")
    users = list(db.users.find({}, {"display_name": 1, "contact_email": 1, "_id": 1}))
    
    if users:
        print(f"Found {len(users)} users:")
        for user in users:
            print(f"  - {user.get('display_name', 'NO NAME')} (ID: {user['_id']})")
    else:
        print("No users found in database")
        
    print("\n=== Available collections ===")
    collections = db.list_collection_names()
    print(f"Collections: {collections}")
    
except Exception as e:
    print(f"Connection failed: {e}")
    print("Make sure MongoDB is running on localhost:27017")

=== Connection successful ===
Database name: datalabvue

=== All users in database ===
Connection failed: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 1000.0ms, connectTimeoutMS: 1000.0ms), Timeout: 1.0s, Topology Description: <TopologyDescription id: 68889282b5a686d5fe980dcb, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 1000.0ms, connectTimeoutMS: 1000.0ms)')>]>
Make sure MongoDB is running on localhost:27017


In [5]:
#TODO Fix this so it works
# Example API call to completely remove a user
from pymongo import MongoClient

# Use localhost instead of 'database' hostname - the default configuration in pydatalab.config.ServerConfig uses "mongodb://localhost:27017/datalabvue" as the default MONGO_URI.
client = MongoClient("mongodb://localhost:27017/datalabvue")
db = client.get_database()

# Delete a user by their ID
# result = db.users.delete_one({"_id": "user_id_here"})
# print(f"Deleted {result.deleted_count} user(s)")

# Or delete by display_name (not username)
result = db.users.delete_one({"display_name": "Xi Yang"})
print(f"Deleted {result.deleted_count} user(s)")

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 68889476b5a686d5fe980dcc, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [2]:
result

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [ ]:
#TODO Example API call to deactivate a user
import requests

response = requests.patch(
    "http://sce-chem-c01894.chem.ed.ac.uk:5001/users/<user_id>",
    json={"account_status": "deactivated"},
    headers={"Authorization": "Bearer <admin_api_key>"}
)